# Finetuning Medical Claim Classifier Problem: Starter Code


## Connecting to Google Drive (Optional)

We recommend you to save files in your Google Drive to avoid having to download every time you run this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# Change the code below to whichever folder you would like your directory to be at. Remove # and run.
path = '/content/gdrive/MyDrive/medclaim_detection'
%cd {path}

Mounted at /content/gdrive/
/content/gdrive/MyDrive/medclaim_detection


## Import libraries

We will use HuggingFace's `transformers` libraries to load a pre-trained DistilBert model.

In [ ]:
import argparse
import logging
import os
import re
from typing import Dict
import itertools

import numpy as np
import pandas as pd
import nltk
import torch
import torch.nn as nn
from datasets import Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    Trainer,
    TrainingArguments,
)
from torchvision.ops import sigmoid_focal_loss

Use logging for storing various pieces of information during training.

In [ ]:
import sys
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
def download_data(args):
    """Checks if data file exists and downloads it if not."""
    file_path = os.path.join(args.data_dir, 'Twitter.csv')
    url = 'https://raw.githubusercontent.com/LCS2-IIITD/LESA-EACL-2021/main/data/Twitter.csv'

    # Check if data_dir exists, if not, make folder
    if not os.path.exists(args.data_dir):
        os.makedirs(args.data_dir)
        logger.info('Created directory: %s', args.data_dir)

    if not os.path.exists(file_path):
        logger.info('Downloading Twitter.csv from %s', url)
        try:
            r = requests.get(url, allow_redirects=True)
            with open(file_path, 'wb') as f:
                f.write(r.content)
            logger.info('Downloaded Twitter.csv to %s', file_path)
        except requests.exceptions.RequestException as e:
            logger.error('Error downloading file: %s', e)
            raise

    args.data_path = file_path

In [ ]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """Apply light preprocessing and add `clean_text` column."""
    df = df.copy()
    df['hashtag'] = df['tweet_text'].apply(lambda x: re.findall(r'#(\w+)', str(x)))
    df['clean_text'] = df['tweet_text'].apply(lambda x: re.sub(r'http\S+|www\S+|@[\S]+', '', str(x)))

    tokenizer = TweetTokenizer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def tok_and_clean(text: str):
        toks = tokenizer.tokenize(text.lower())
        toks = [lemmatizer.lemmatize(t) for t in toks if t.isalpha()]
        toks = [t for t in toks if t not in stop_words]
        return ' '.join(toks)

    df['clean_text'] = df['clean_text'].apply(tok_and_clean)
    return df

In [ ]:
def compute_metrics(pred) -> Dict[str, float]:
    logits, labels = pred
    # handle logits shape (N,1) or (N,)
    logits = np.asarray(logits)
    if logits.ndim > 1:
        logits = logits.reshape(-1)
    probs = 1.0 / (1.0 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    acc = accuracy_score(labels, preds)
    precision_arr, recall_arr, f1_arr, _ = precision_recall_fscore_support(labels, preds, labels=[0, 1], zero_division=0)

    metrics = {
        'accuracy': float(acc),
        'precision_class_0': float(precision_arr[0]),
        'precision_class_1': float(precision_arr[1]),
        'recall_class_0': float(recall_arr[0]),
        'recall_class_1': float(recall_arr[1]),
        'f1_class_0': float(f1_arr[0]),
        'f1_class_1': float(f1_arr[1]),
    }
    metrics['f1'] = float(np.mean([metrics['f1_class_0'], metrics['f1_class_1']]))
    return metrics

In [ ]:
def load_pretrained_and_finetune(args):
    nltk.download('stopwords')
    nltk.download('wordnet')

    os.makedirs(args.output_dir, exist_ok=True)

    logger.info('Loading data from %s', args.data_path)
    df = pd.read_csv(args.data_path)
    if 'tweet_text' not in df.columns or 'claim' not in df.columns:
        raise ValueError("Input CSV must contain 'tweet_text' and 'claim' columns")

    # print(df.head(10)) # uncomment to print first 10 rows
    df = preprocess(df)
    df = df[['clean_text', 'claim']].rename(columns={'claim': 'labels'})

    # Q2: split the code in train, val and eval (test) sets stratified by classes
    # BEGIN YOUR CODE HERE (~2 lines)
    # END YOUR CODE HERE

    tokenizer = DistilBertTokenizerFast.from_pretrained(args.model_name)

    def tokenize_batch(batch):
        return tokenizer(batch['clean_text'], truncation=True, padding='max_length', max_length=args.max_length)

    # Q3: Convert sets to HuggingFace Dataset and tokenize using function tokenize_batch
    # Use variable names: train_ds, val_ds, eval_ds
    # BEGIN YOUR CODE HERE (~6 lines)
    # END YOUR CODE HERE

    # Kepe only the necessary columns in each dataset
    keep_cols = ['input_ids', 'attention_mask', 'labels']
    train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in keep_cols]).with_format('torch')
    val_ds = val_ds.remove_columns([c for c in val_ds.column_names if c not in keep_cols]).with_format('torch')
    test_ds = test_ds.remove_columns([c for c in test_ds.column_names if c not in keep_cols]).with_format('torch')

    # We'll create models inside the grid loop; keep tokenizer ready

    # Device selection: prefer CUDA, then Apple MPS, then CPU
    if hasattr(torch, 'cuda') and torch.cuda.is_available():
        device = 'cuda'
    else:
        # MPS (Apple Silicon) support
        try:
            if getattr(torch, 'has_mps', False) and torch.backends.mps.is_available():
                device = 'mps'
            else:
                device = 'cpu'
        except Exception:
            device = 'cpu'

    # Ensure Trainer knows whether to use CUDA
    no_cuda = False if device in ('cuda', 'mps') else True

    # Note: model instances are created per-trial inside the grid search loop below.

    # Grid search over learning rate (no focal gamma needed)
    best_score = -float('inf')
    best_params = None

    # Q4: Implement grid search for at least one hyperparameter

    # Build hyperparameter lists
    # BEGIN YOUR CODE HERE (~1-7 lines)
    # PRO TIP: you can use itertools.product to create a master list of all combos for grid search

    for # COMPLETE THIS LINE
        logger.info('WHAT DO YOU WANT TO LOG?')
        # instantiate fresh pretrained model for each trial
        # NOTE: if num_labels=2, CrossEntropy is assumed by default Trainer;
        # if num_labels=1, you will have to implement a CustomTrainer to override MSE as loss
        model = # COMPLETE THIS LINE

        # Freeze base params and unfreeze classifier layers
        for name, p in model.named_parameters():
            p.requires_grad = False
        for name, p in model.named_parameters():
            if # COMPLETE THIS LINE
                p.requires_grad = True

        # trial output dir
        trial_output_dir = os.path.join(args.output_dir, f'CHOOSE_EXP_NAME')
        os.makedirs(trial_output_dir, exist_ok=True)

        training_args = TrainingArguments(
            # INCLUDE RELEVANT ARGUMENTS (~5-10 LINES)
            no_cuda=no_cuda,
            report_to='none',
        )

        # initialize the Trainer (~5 lines) or CustomTrainer (~6-8 lines)
        trainer = # COMPLETE THIS LINE
        )

        # move model to device if possible
        # BEGIN YOUR CODE HERE (~1-4 lines)
        # END YOUR CODE HERE

        # train, evaluate, compute score, update best score
        # BEGIN YOUR CODE HERE (~6-9 lines)
        # END YOUR CODE HERE

    # END YOUR CODE HERE

    if best_params is None:
        raise RuntimeError('Grid search failed to produce any candidate best params')

    logger.info('Best hyperparameters from grid search: %s (score=%s)', best_params, best_score)

    # Q5: Final training on train+val with best hyperparameters
    # instantiate pretrained model and freeze appropriate layers
    # BEGIN YOUR CODE HERE (~6 lines)
    # END YOUR CODE HERE

    # combine train_ds and val_df for final training (i.e., original train_df)
    # BEGIN YOUR CODE HERE (~4 lines)
    # END YOUR CODE HERE

    # set the training arguments
    # BEGIN YOUR CODE HERE (~9-14 lines)
    # END YOUR CODE HERE

    # initialize the Trainer (~5 lines) or CustomTrainer (~6-8 lines)

    # move model to device if possible
    # BEGIN YOUR CODE HERE (~1-4 lines)
    # END YOUR CODE HERE

    # train, evaluate, save model
    # BEGIN YOUR CODE HERE (~3 lines)
    # END YOUR CODE HERE
    trainer.train()
    trainer.save_model(args.output_dir)

In [ ]:
# if __name__ == '__main__':
parser = argparse.ArgumentParser()
parser.add_argument('--data-dir', type=str, required=True, help='Path to CSV with tweet_text and claim columns')
parser.add_argument('--output-dir', type=str, default='finetuned', help='Where to save model and tokenizer')
parser.add_argument('--model-name', type=str, default='distilbert-base-uncased', help='Pretrained model name')
# Q1. Add relevant arguments
# BEGIN YOUR CODE HERE (~5-15 lines)
# END YOUR CODE HERE

args = parser.parse_args(['--data-dir','data'] # EDIT THIS LINE TO PLAY WITH NON-DEFAULT ARGS
print(args)

# download file if it doesn't exist yet
download_data(args)

# load pre-trained and finetune
load_pretrained_and_finetune(args)

## Q6: Based on the item above, discuss the performance of the model, any challenges faced during fine-tuning, and potential improvements that can be made to further improve accuracy.

ENTER YOUR ANSWER HERE